# DX 704 Week 5 Project

This week's project will test your understanding of this week's concepts by asking you to simulate various algorithms by hand.
You will apply search, minimax and dynamic programming concepts to solve a variety of small planning problems.

The full project description, a template notebook and supporting materials are available on GitHub: [Project 5 Materials](https://github.com/bu-cds-dx704/dx704-project-05).

## Example Code

This week's assignment does not involve any coding.

## Part 1: Searching vs Games

Consider the state space illustrated below.
Each terminal state state is marked with a reward for reaching that state.
Each non-terminal state has two possible actions represented by the two outgoing arrows to later (lower) states.
The only rewards are for reaching the terminal states, there are no diminishing returns (i.e. $\gamma=1$), and there is no randomness so actions may be freely chosen.

![](https://github.com/bu-cds-dx704/dx704-project-05/blob/main/part1.png?raw=true)

Solve for the value of each non-terminal state according to the following three scenarios.

1. Search: There is one agent that picks all actions with the goal of maximizing the final reward.
2. Minimax: There are two agents P1 and P2. P1 controls the actions for the 1st and 3rd rows (i.e. the states marked A and D-G) while P2 controls the actions for the 2nd and 4th rows (i.e. the states B-C and H-J). P1 seeks to maximize the final reward, and P2 seeks to minimize the final reward.
3. Maximin: P1 and P2 control the same states as before, but P1 seeks to minimize the final reward, and P2 seeks to maximize the final reward.

Save your results in a file "values-1.tsv" with the column state with label A-J and columns search_value, minimax_value, and maximin_value that respectively correspond to the three scenarios.

Hint: Print out the image above and compute the values by hand in a bottom up fashion.

Submit the file "values-1.tsv" in Gradescope.

In [4]:
import pandas as pd, os

# 1. search
vals_search = {'A': 20,'B': 20,'C': 1,'D': 20,'E': 5,'F': 1,'G': 1,'H': 9,'I': 20,'J': 3}
# 2. minimax
vals_minimax = {'A': 1,'B': 1,'C': 1,'D': 1,'E': 5,'F': 1,'G': 1,'H': -1,'I': 1,'J': 2}
# 3. maximin
vals_maximin = {'A': 0,'B': 9,'C': 0,'D': 9,'E': 3,'F': -1,'G': 0,'H': 9,'I': 20,'J': 3}

states = list("ABCDEFGHIJ")
df = pd.DataFrame({
    "state": states,
    "search_value": [vals_search[s] for s in states],
    "minimax_value": [vals_minimax[s] for s in states],
    "maximin_value": [vals_maximin[s] for s in states],
})

df.to_csv("values-1.tsv", sep="\t", index=False)
df

,state,search_value,minimax_value,maximin_value
0,A,20,1,0
1,B,20,1,9
2,C,1,1,0
3,D,20,1,9
4,E,5,5,3
5,F,1,1,-1
6,G,1,1,0
7,H,9,-1,9
8,I,20,1,20
9,J,3,2,3


## Part 2: Picking Up Sticks

The state space illustrated below corresponds to a variation of the game [Nim](https://en.wikipedia.org/wiki/Nim).
States labeled with a prefix of "p1_" correspond to states where player P1 chooses the action while states labeled with a prefix of "p2_" correspond to states where player P2 chooses the action.
The number in the suffix is the number of "sticks" remaining.
The players take turns choosing actions, and each action corresponds to removing one or two sticks.
When there are no more sticks, the player who would have picked an action loses.


![](https://github.com/bu-cds-dx704/dx704-project-05/blob/main/part2.png?raw=true)

For example, from the state labeled "p1_1", there is one stick left, player P1 removes the last stick, and player P2 loses.
The loss for P2 is represented by a final reward of +1.
A loss for P1 is represented by a final reward of -1.
Player P1 tries to maximize the final reward, and player P2 tries to minimize the final reward.

Solve for the value of each of the non-terminal states.
Save the results in a file "values-2.tsv" with columns state and value.

Submit the file "values-2.tsv" in Gradescope.

In [5]:
# working backwards
vals = {
    "p1_1":  1,
    "p2_1": -1,
    "p1_2":  1,
    "p2_2": -1,
    "p1_3": -1,
    "p2_3":  1,
    "p1_4":  1,
    "p2_4": -1,
    "p1_5":  1,
    "p2_5": -1,
}

df = pd.DataFrame({"state": list(vals.keys()), "value": list(vals.values())})
df.to_csv("values-2.tsv", sep="\t", index=False)
df

,state,value
0,p1_1,1
1,p2_1,-1
2,p1_2,1
3,p2_2,-1
4,p1_3,-1
5,p2_3,1
6,p1_4,1
7,p2_4,-1
8,p1_5,1
9,p2_5,-1


## Part 3: Searching a Maze

Consider the following maze.

![](https://github.com/bu-cds-dx704/dx704-project-05/blob/main/part3.png?raw=true)

State C is a terminal state giving reward +100.
The remaining states have a reward of -1 when they are reached.
So moving to state F has a value of +99 do to the reward of -1 at state F and the optimal action of moving to state C for the reward of +100 afterwards.

Compute the values for states A-J and S and save them in a file "values-3.tsv" with columns state and value.

Submit "values-3.tsv" in Gradescope.

In [8]:
from collections import deque

# maze
adj = {
    "A": ["S"],
    "S": ["A", "B"],
    "B": ["S", "D"],
    "D": ["B", "E", "G"],
    "E": ["D", "H"],
    "G": ["D", "I"],
    "H": ["E", "I"],
    "I": ["H", "G", "J"],
    "J": ["I", "F"],
    "F": ["J", "C"],  # C is terminal
}

# only need distances to F bc F -> C is the only way into C
target = "F"

dist = {target: 0}
q = deque([target])

# reverse edges so i can compute distance to target
rev = {s: [] for s in adj}
for s, nbrs in adj.items():
    for t in nbrs:
        if t in rev:
            rev[t].append(s)

while q:
    cur = q.popleft()
    for prev in rev[cur]:
        if prev not in dist:
            dist[prev] = dist[cur] + 1
            q.append(prev)

vals = {s: 100 - dist[s] for s in dist if s != "C"}  # exclude C
df = pd.DataFrame({"state": sorted(vals.keys()), "value": [vals[s] for s in sorted(vals.keys())]})
df.to_csv("values-3.tsv", sep="\t", index=False)
df


,state,value
0,A,93
1,B,95
2,D,96
3,E,96
4,F,100
5,G,97
6,H,97
7,I,98
8,J,99
9,S,94


## Part 4: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [9]:
content = (
    "Acknowledgments\n\n"
    "I used the DX704 course-provided code examples as a reference when developing "
    "the code for this assignment.\n\n"
    "I referenced the following notebook from the course repo:\n"
    "https://github.com/bu-cds-omds/dx704-examples/blob/main/week01/"
    "chart_risk_return_performance_of_efficient_portfolios.ipynb\n"
    "I also used reddit and youtube.\n"
    "Lastly, my partner is a robotics engineer at Google\n"
    "and I double-checked my code with him."
)

with open("acknowledgments.txt", "w") as f:
    f.write(content)